# Illustrates how to stream USB traffic:
It isn't necessary to wait for the capture to be finished before we start reading; we can even start reading before the capture starts.

What this means is that for targets that don't generate traffic faster than we can read it (e.g. LS and FS targets), the capture size isn't limited by the PhyWhisperer's internal storage; our capture size is theoretically unlimited!

(Captured data is read from the PhyWhisperer at up to 40 Mbps. The exact rate depends on the nature of the target's USB traffic, because there is overhead in the PhyWhisperer capture for timestamp and USB status data.)

For HS targets, we can't read the captured data as fast as it's generated, so the capture size remains limited, but it's still possible to capture more than what the PhyWhisperer's internal storage can hold.

### Connect:

In [ ]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=True)
phy.set_power_source("host")

### Set capture parameters:
Compared to the [basic sniffing tutorial](sniff.ipynb), we'll do a few things differently here:

1. In order to capture more than what the PhyWhisperer can store, we must begin reading the capture as soon as possible.

2. To facilitate this, we'll manually set the USB target speed, instead of using the autodetect feature. This allows the capture to start immediately after the target powers on.

3. We'll set the capture size to be unlimited with `set_capture_size(0)`. It could also be set to a fixed number of events (max 2^24-1).

4. We'll stop reading capture data after 5 seconds with `read_capture_data(timeout=5)`.

In [ ]:
import time
raw = []

phy.set_power_source("off")
time.sleep(0.5)
phy.reset_fpga()
phy.set_usb_mode('FS')

phy.set_capture_size(0xffff)
phy.arm()
phy.set_trigger(enable=False)
phy.set_pattern(pattern=[0x2d, 0x00], mask=[0xff, 0xff])

phy.set_power_source("5V")

raw = phy.read_capture_data(timeout=5)

### Check how many entries were captured:

In [ ]:
print('Captured %d entries.' % len(raw))
assert len(raw) > 8189, "Oops, didn't capture more than what the PhyWhisperer's internal memory can store. Something went wrong."

### Check the capture memory status:
Here, because the capture size was set to "unlimited", the PhyWhisperer will overflow shortly after we stop reading from it, so an overflow error is expected.
To avoid this, change the capture size to a number smaller than the number of events you captured here.

In [ ]:
phy.check_fifo_errors()

### Interpret the captured data:
The pattern match byte which triggered the captured isn't recorded; let's add it back it so that all of the captured USB data can be properly interpreted:

In [ ]:
phy.addpattern = True

Then we split the raw captured data and timestamps into packets (if the capture is very large this could take a while):

In [ ]:
packets = phy.split_packets(raw)
print('Captured %d packets.' % len(packets))

Now we can use ViewSB to interpret the packets. Careful, the output can be VERY long, so we'll just print the first few packets:

In [ ]:
phy.print_packets(packets[:20])

In [ ]:
phy.close()